<a href="https://colab.research.google.com/github/lonespear/aerogel/blob/main/transfer_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

# Load pre-trained VGG16 model (without top layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [2]:
def create_segmentation_model(base_model):
    inputs = base_model.input
    # Add deconvolution and upsampling layers
    x = base_model.output

    # Decoder block 1
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)

    # Decoder block 2
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)

    # Decoder block 3
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)

    # Final layer for segmentation
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(x)

    # Create the full model
    model = models.Model(inputs, outputs)
    return model

segmentation_model = create_segmentation_model(base_model)


In [3]:
for layer in base_model.layers:
    layer.trainable = False


In [4]:
segmentation_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create ImageDataGenerator for augmentation and preprocessing
data_gen_args = dict(rotation_range=15,
                     width_shift_range=0.05,
                     height_shift_range=0.05,
                     shear_range=0.05,
                     zoom_range=0.05,
                     horizontal_flip=True,
                     fill_mode='nearest')

image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

# Prepare generators for training
image_generator = image_datagen.flow_from_directory('path_to_images', target_size=(256, 256), class_mode=None, batch_size=16)
mask_generator = mask_datagen.flow_from_directory('path_to_masks', target_size=(256, 256), class_mode=None, batch_size=16)

# Combine generators
train_generator = zip(image_generator, mask_generator)


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_images'